# Packages


In [ ]:
!pip install seaborn
!pip install matplotlib
!pip install -U scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import tensorflow as tf

# Surrogate Model Building


In [ ]:
def sigmoid(x):
    return (1 / (1 + math.exp(-x)))
def epsilons ():
    return np.random.normal(loc = 0,scale = 0.5)
def para_c1():
    return np.random.normal(loc=-2,scale=1)
def para_c2():
    return np.random.normal(loc=2,scale=1)
def para_cw_2():
    return np.random.normal(loc=1,scale=1)

def build_w1(u1,u2):
    return sigmoid(para_c1()+u1+para_c2()+u2+epsilons())

def build_w2(w1,d):
    temp = 1.0
    for i in range(d):
        p = para_cw_2()
        temp += (2*w1[i] - 1)*p
    return sigmoid(-temp + epsilons())

def build_x(u1,w2):
    return sigmoid(u1 - 4*w2 + 2 + epsilons())

def build_y(u2,x):
    return sigmoid(0.5*u2-2*x+1+epsilons())

def build_causal_model(n_samples,d):
    scm_numpy = np.empty((n_samples,d+3))
    col_name = list()
    for i in range(n_samples):
        u1 = np.random.normal(loc=0,scale=1)
        u2 = np.random.normal(loc=0,scale=1)
        w1_s = list()
        for j in range(d):
            if(i == 0):
                col_name.append('w1,'+str(j))
            w1 = np.random.binomial(n=1,p=build_w1(u1, u2))
            scm_numpy[i][j] = w1
            w1_s.append(w1)
        w2 = np.random.binomial(n=1,p=build_w2(w1_s, d))
        x = np.random.binomial(n=1,p=build_x(u1, w2))
        y = build_y(u2, x)
        scm_numpy[i][d] = w2
        scm_numpy[i][d+1] = x
        scm_numpy[i][d+2] = y
        if(i == 0):
            col_name.extend(['w2','x','y'])
    # df = pd.DataFrame()
    return scm_numpy ,col_name


# DO on Surrogate(Ground Truth)


In [ ]:
def causal_do(scm,new_x):
    print('bbbb',scm)
    scm_copy = scm.copy()
    scm_cols = np.size(scm_copy,axis=1)
    scm_rows = np.size(scm_copy,axis=0)
    print('scm_cols->',scm_cols)
    scm_copy[:,scm_cols-2] = new_x
    for i in range(scm_rows):
        u2 = np.random.normal(loc=0,scale=1)
        x = scm_copy[i,scm_cols-2]
        scm_copy[i][scm_cols-1] = build_y(u2, x)

    print('modified')
    print(scm_copy)
    return scm_copy


def estimate_conditional_expectation1(df):
    scm_cols = np.size(df,axis=1)
    scm_rows = np.size(df,axis=0)
    a = df[:,scm_cols-2] == 0
    b = df[:,scm_cols-2] == 1
    y = df[:,scm_cols-1]

    p = np.mean(y[b])
    q = np.mean(y[a])
    delta = p-q
    return p,q,delta
def ab_test1(scm, x, y, n):
    n_a = int(n / 2)
    n_b = n - n_a
    set_variable = np.array([0]*n_a + [1]*n_b)
    scm = causal_do(scm,set_variable)
    return estimate_conditional_expectation1(scm)


# Weighted


In [ ]:
def build_zs_t(d):
    zs_unique = np.empty((d,2**d),int)
    for i in range(d):
        d_bar = 2**d/(2**(i+1))
        d1 = 0
        pp=1
        for j in range(2**d):
            if(pp == 1):
                zs_unique[i][j] = 0
                d1+=1
                if(d1==d_bar):
                    pp=-1
                    d1=0
            elif(pp == -1):
                zs_unique[i][j] = 1
                d1+=1
                if(d1 == d_bar):
                    pp=1
                    d1=0

    zs_t = zs_unique.transpose()
    return zs_t

In [ ]:
def p_w_2(fd):
    df_copy = fd['w2'].copy()
    size = fd.shape[0]
    pw2 = np.zeros(2)
    prob_w2 = np.zeros(size)
    for i in [0,1]:
        nm1 = df_copy == i
        pw2[i] = nm1.mean()
    for i in range(size):
        if df_copy[i] == 0:
            prob_w2[i] = pw2[0]
        else :
            prob_w2[i] = pw2[1]
    return prob_w2
def p_w1_w2(df):
    df_copy = df.copy()
    d=df_copy.shape[1]-3
    df_size = df.shape[0]
    proba_z_x = np.ones(df.shape[0])
    df_w1 = df_copy.iloc[:,0:d]
    x_pr = df['w2']
    X=df_w1.values.reshape(-1, d)
    y=df['w2']
    clf = LogisticRegression(random_state=0).fit(X, y)
    predict = clf.predict_proba(X)
    for j in range(df_size):
      if ( x_pr[j] == 0):
        proba_z_x[j] *= predict[j,0]
      else :
        proba_z_x[j] *= predict[j,1]
    return proba_z_x

In [ ]:
def weightGenerator(df):
  data1 = p_w_2(df)
  data2 = p_w1_w2(df)
  data_prob = np.divide(data1,data2)
  return data_prob

In [ ]:
def cwoBeta(X,y,weight,beta):
  Beta1 = LinearRegression().fit(X,y,sample_weight=weight)
  test_x=np.array([[0,0], [1,1]]).reshape((-1, 2))
  y_pred=Beta1.predict(test_x)

  return y_pred

In [ ]:
def weightedATE(df):
  data=df.copy()
  d=data.shape[1]-3
  X=data.iloc[:,d:d+2].values.reshape(-1, 2)
  Y=data['y'].values.reshape(-1, 1)
  data['weight'] = weightGenerator(df)
  ate=cwoBeta(X,Y,data['weight'],1)
  return ate[1][0],ate[0][0],ate[1][0]-ate[0][0]

# Neural Network

In [ ]:
hpDict={'conv_0_units': 40,
 'dropout': 0.2,
 'dropout_0_': 0.25,
 'input_units': 60,
 'learning_rate': 0.00925824604021082,
 'n_layers': 1}

In [ ]:
def nnBeta(X,y,testX,weight,hp):
  Beta2 = tf.keras.models.Sequential()
  Beta2.add(tf.keras.layers.Dense(hp['input_units'],activation='relu',input_shape=(2,)))
  Beta2.add(tf.keras.layers.Dropout(hp['dropout']))
  for i in range(hp['n_layers']):
    Beta2.add(tf.keras.layers.Dense(hp[f'conv_{i}_units'], activation='relu'))
    Beta2.add(tf.keras.layers.Dropout(hp[f'dropout_{i}_']))

  Beta2.add(tf.keras.layers.Dense(1, activation='linear'))
  Beta2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp['learning_rate']), loss='mae')

  early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=40, verbose=1, mode='auto', restore_best_weights=True)
  history=Beta2.fit(X,y,epochs=1000,
                    validation_split=0.2,
                    shuffle = True,
                    callbacks=[early_stop],
                    verbose=0
                    ,sample_weight=weight
                    )


  y_pred = Beta2.predict(testX)
  return y_pred

In [ ]:
def NeuralATE(df,hp):
  data=df.copy()
  d=data.shape[1]-3
  X=data.iloc[:,d:d+2].values.reshape(-1, 2)
  Y=data['y'].values.reshape(-1, 1)
  sampleWeight = weightGenerator(df)
  test=np.array([[1,0], [0,1]])
  test_x=pd.DataFrame(test,columns=['a','b'])
  XX=test_x.values.reshape(-1, 2)
  ate = nnBeta(X,Y,XX,sampleWeight,hp)


  return ate[1][0],ate[0][0],ate[1][0]-ate[0][0]


# Hyper Parameter Tuning

In [ ]:
!pip install keras-tuner==1.0.0
import kerastuner

In [ ]:
from kerastuner.tuners import RandomSearch
import time
LOG_DIR = f"{int(time.time())}"

In [ ]:
def tempNN(hp):
  Beta2 = tf.keras.models.Sequential()
  Beta2.add(tf.keras.layers.Dense(hp.Int('input_units',
                                         min_value=10,
                                         max_value=100,
                                         step=10), activation='relu',input_shape=(2,)))
  Beta2.add(tf.keras.layers.Dropout(hp.Float('dropout',
                                             min_value=0.0,
                                             max_value=0.4,
                                             default=0.1,
                                             step=0.05,)))
  for i in range(hp.Int('n_layers', 0, 4)):
    Beta2.add(tf.keras.layers.Dense(hp.Int(f'conv_{i}_units',
                                           min_value=10,
                                           max_value=100,
                                           step=10), activation='relu'))
    Beta2.add(tf.keras.layers.Dropout(hp.Float(f'dropout_{i}_',
                                             min_value=0.0,
                                             max_value=0.4,
                                             default=0.1,
                                             step=0.05,)))

  Beta2.add(tf.keras.layers.Dense(1, activation='linear'))
  Beta2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate',
                                                                            min_value=1e-5,
                                                                            max_value=1e-2,
                                                                            sampling='LOG',
                                                                            default=1e-3)), loss='mse')

  return Beta2

In [ ]:
tuner = RandomSearch(
    tempNN,
    objective='val_loss',
    max_trials=5,  # how many model variations to test?
    executions_per_trial=4,  # how many trials per variation? (same model could perform differently)
    directory=LOG_DIR)

In [ ]:
data=fd.sample(n=500).reset_index(drop=True)
X=data.iloc[:,D:D+2].values.reshape(-1, 2)
Y=data['y'].values.reshape(-1, 1)
sampleWeight = weightGenerator(data)
early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=20, verbose=1, mode='auto', restore_best_weights=True)

In [ ]:
tuner.search(X,Y,epochs=500,
             validation_split=0.2,
             shuffle = True,
             callbacks=[early_stop],
             verbose=2
             ,sample_weight=sampleWeight
         )

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
tuner.get_best_models()[0].summary()

In [ ]:
def doDo(d):
  N=10000000
  scm_np ,col_name= build_causal_model(N, d)
  mu1,mu0,muATE=ab_test1(scm_np,'x','y',N)
  df = pd.DataFrame(scm_np,columns=col_name)
  return df,mu1,mu0

In [ ]:
D=7

In [ ]:
fd,mu1,mu0=doDo(D)
print("Estimated ATE: {:.3f}".format(mu1-mu0))
print(fd)
print("mu(1): {:.3f}".format(mu1))
print("mu(0): {:.3f}".format(mu0))

# Saving and loading pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
fd.to_pickle("drive/MyDrive/Datasets/sg_df_7.pkl")

In [ ]:
string1 = str(mu1) + " " + str(mu0)
print(string1)
f = open("drive/MyDrive/Datasets/sgmu_7.txt","w")
f.write(string1)
f.close()

# Result Generation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
fd = pd.read_pickle("/content/drive/MyDrive/Datasets/sg_df_7.pkl")
print(fd)
f = open('/content/drive/MyDrive/Datasets/sgmu_7.txt','r')
mm = f.read()
mu1,mu0 = mm.split()
mu1=float(mu1)
mu0=float(mu0)
print(mu1,mu0)

In [ ]:
def MAAEgenerator(fd,mu1,mu0):

  muN1,muN0,_=NeuralATE(fd,hpDict)
  muC1,muC0,_=weightedATE(fd)
  maaeN=(abs(muN1-mu1)+abs(muN0-muN0))/2
  maaeC=(abs(muC1-mu1)+abs(muC0-mu0))/2
  return maaeN,maaeC



In [ ]:
N_SAMPLE=500

In [ ]:
for N_SAMPLE in range(4000,10001,500):
  muN=np.zeros(100)
  muC=np.zeros(100)
  for k in range(100):
    # print("K= ",k)
    db=fd.sample(n=N_SAMPLE).reset_index(drop=True)
    muN[k],muC[k]=MAAEgenerator(db,mu1,mu0)
    # print(k,muN[k],muC[k])
  print(N_SAMPLE)
  MAAE_N=np.median(muN)
  print("MAAE NN",MAAE_N)
  MAAE_C=np.median(muC)
  print("MAAE CWO",MAAE_C)
  # print("MAAE NN: {:.5f}".format(MAAE_N))
  # print("MAAE CWO: {:.5f}".format(MAAE_C))